In [ ]:
!pip install -q niyamit-sphere

In [ ]:
!wget "https://raw.githubusercontent.com/Niyam-Projects/sphere/refs/heads/main/examples/HI_Honolulu_UDF_sample.csv"
!wget "https://raw.githubusercontent.com/Niyam-Projects/sphere/refs/heads/main/examples/Oahu_10_withReef.tif"
!wget "https://raw.githubusercontent.com/Niyam-Projects/sphere/refs/heads/main/examples/Oahu_10_withReef.tfw"

In [ ]:
import time
from pathlib import Path
import pandas as pd
from sphere.core.schemas.fast_buildings import FastBuildings
from sphere.flood.analysis.hazus_flood import HazusFloodAnalysis
from sphere.flood.single_value_reader import SingleValueRaster
from sphere.flood.default_vulnerability import DefaultFloodVulnerability

# Define file paths (adjust these paths as necessary)
# The base directory is set to the content folder since the files are located there
base_dir = Path("/content")
buildings_csv = base_dir / "HI_Honolulu_UDF_sample.csv"
tif_file = base_dir / "Oahu_10_withReef.tif"

In [ ]:
# Load buildings data from CSV
buildings = FastBuildings(str(buildings_csv))
display(buildings.gdf.head())

In [ ]:
# Read the depth grid from the TIFF file
depth_grid = SingleValueRaster(str(tif_file))
# Depending on the SingleValueRaster object structure, you might want to display
# some information about the loaded raster, e.g., its extent or values.
# For now, we'll just confirm it's loaded.
print("Depth grid loaded successfully.")

In [ ]:
# Create an instance of the default flood function
flood_function = DefaultFloodVulnerability(buildings, flood_type="R")
print("Flood vulnerability function created.")

In [ ]:
# Create the Hazus flood analyzer instance.
# (Assumes that HazusFloodAnalyzer accepts buildings DataFrame, depth grid, the flood function,
# and geospatial metadata like transform and crs.)
analyzer = HazusFloodAnalysis(
    buildings=buildings,
    vulnerability_func=flood_function,
    depth_grid=depth_grid,
)
print("Hazus flood analyzer created.")

In [ ]:
start_time = time.time()
# Calculate losses using the analysis
analyzer.calculate_losses()  # Expected to return a DataFrame
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Calculation time: {elapsed_time:.6f} seconds")

In [ ]:
# Save the results to a CSV file
results_csv = base_dir / "flood_losses.csv"
buildings.gdf.to_csv(results_csv, index=False)

print(f"Flood analysis complete. Results saved to: {results_csv}")
display(buildings.gdf.head())

The results will be in the flood_losses.csv file.  Not every row will have estimates as some of the sample data is outside the flood extent.